In [3]:
import pandas as pd
import psycopg2
import numpy as np
import re
from sqlalchemy import create_engine, text as sql_text
import datetime
from ydata_profiling import ProfileReport

# Projeto Analytics Engineering - Inside Airbnb

## PostgreSQL - Exportação dos dados "raw"

Conjunto de dados:  "Inside Airbnb"

Passo a Passo:
1. Vá no link https://insideairbnb.com/get-the-data
2. Procure por "Rio de Janeiro"
3. Baixe os 3 .csv: "Listing", "Reviews" e Calendar" 

0bs: Caso queira pular estes passos, basta utilizar os .csvs que já estão nessa pasta: https://drive.google.com/drive/folders/1pwCpVsMu5si7BX1lzWX3tipP3fAPuoX3?usp=drive_link

4. Crie uma pasta dentro desse repositorio como 'raw_data' e adicione os 3 .csv

### Conexão com PostgreSQL

In [4]:
# James:
#conn = psycopg2.connect(dbname='airbnb_project', user='postgres', host='localhost', password='123456', port="5432")

# Marcela: 
conn = psycopg2.connect(dbname='myDBProject', user='postgres', host='localhost', password='0125698741', port="5433")
db_params = {
    'dbname': 'myDBProject',
    'user': 'postgres',
    'host': 'localhost',
    'password': '0125698741',
    'port': '5433'
}
db_url = f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}'


### Criação das bases 'raw's
- Crie um banco de dados PostgreSQL para armazenar os dados brutos das 3 tabelas ("Listing", "Reviews" e Calendar") 

In [14]:
engine = create_engine(db_url)
df_listings_raw = pd.read_csv("raw_data/listings.csv")
df_reviews_raw = pd.read_csv("raw_data/reviews.csv")
df_calendar_raw = pd.read_csv("raw_data/calendar.csv")

In [15]:
df_listings_raw.to_sql('listings_raw', engine, if_exists='replace', index=False)
df_reviews_raw.to_sql('reviews_raw', engine, if_exists='replace', index=False)
df_calendar_raw.to_sql('calendar_raw', engine, if_exists='replace', index=False)

595

In [16]:
def load_data_from_table(table_name):
    query = f"SELECT * FROM {table_name};"
    df = pd.read_sql_query(query, conn)
    return df
df = load_data_from_table('listings_raw')
df.head(2)

/var/folders/8k/smxjq0q17kz2976yb3t29z6r0000gn/T/ipykernel_51914/2733534399.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,17878,https://www.airbnb.com/rooms/17878,20231226034138,2023-12-27,city scrape,Condo in Rio de Janeiro · ★4.70 · 2 bedrooms ·...,None,This is the one of the bests spots in Rio. Bec...,https://a0.muscache.com/pictures/65320518/3069...,68997,...,4.91,4.77,4.67,None,f,1,1,0,0,1.90
1,25026,https://www.airbnb.com/rooms/25026,20231226034138,2023-12-27,city scrape,Rental unit in Rio de Janeiro · ★4.72 · 1 bedr...,None,Copacabana is a lively neighborhood and the ap...,https://a0.muscache.com/pictures/a745aa21-b8dd...,102840,...,4.92,4.84,4.60,None,f,1,1,0,0,1.67


## DBT - Inicialização

Passo a Passo:
1) Instale o DBT
- python -m pip install dbt-postgres
- pip install dbt


2) Vá até sua pasta "dbt" e rode o comando:
3) dbt init analytics_eng_airbnb 

OBS: você pode pular os passos 2 e 3 utilizando a pasta "dbt" dentro do arquivo do Github


# Camadas

## Bronze

Verifique os arquivos criados na pasta dbt/analytics_eng_airbnb/models/bronze
- source.yml com os sources de "listing_raw", "reviews_raw", "calendar_raw"

- models.yml com informações e verificações sobre as novas tabelas que serão criadas: "listing_bronze", "reviews_bronze", "calendar_bronze". Nesse arquivo, executamos:
    - Verificamos se ID em "listings" é unique e not null
    - Verificamos as colunas "date" são do tipo data
    - Adicionamos referencias de "foreign key" para listings_id

- arquivos .sql para cada uma das 3 tabelas

Para executar as regras criadas e importar as tabelas _bronze para o seu PostgreSQL, entre na pasta dbt/analytics_eng_airbnb e rode o comando:

- dbt run --models bronze

## Silver

Verifique os arquivos criados na pasta dbt/analytics_eng_airbnb/models/silver
- source.yml com os sources de "listing_bronze", "reviews_bronze", "calendar_bronze"

- models.yml com informações e verificações sobre as novas tabelas que serão criadas: "listing_silver", "reviews_silver", "calendar_silver". Nesse arquivo, executamos:
    - X Y Z (Aqui, fale sobre qualidade de dados: integridade, precisão e consistencia para os dados da camada). Fale também quais métricas foram selecionadas para continuar em cada lista e o porque. E também sobre os testes feitos (garantindo que a camada atenda as regras de negocio e requisitos de qualidade). Utilize as colunas selecionadas por Andre, explique porque escolheram elas e adicione os testes ">0" (e outros que tem lá).

- arquivos .sql para cada uma das 3 tabelas

Para executar as regras criadas e importar as tabelas _silver para o seu PostgreSQL, entre na pasta dbt/analytics_eng_airbnb e rode o comando:

- dbt run --models silver

## Gold

Verifique os arquivos criados na pasta dbt/analytics_eng_airbnb/models/gold
- source.yml com os sources de "listing_silver", "reviews_silver", "calendar_silver"

- models.yml com informações e verificações sobre as novas tabelas que serão criadas:  "listing_gold", "reviews_gold", "calendar_gold". Nesse arquivo, executamos:
    - X Y Z (Aqui, fale sobre agregações aplicadas - quais fora e porquê. Nessa camada testes não precisam ser executados (acho)).

- arquivos .sql para cada uma das 3 tabelas

Para executar as regras criadas e importar as tabelas _gold para o seu PostgreSQL, entre na pasta dbt/analytics_eng_airbnb e rode o comando:

- dbt run --models gold